<a href="https://colab.research.google.com/github/deepk2001/Duolingo-Reviews-Data-Analysis/blob/main/Practicum_Duolingo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

We have chosen **Duolingo** app on Google Play Store for our Practicum for the following reasons:

1. Popular app, therefore has an abundance of reviews and users
2. Familiarity, the team is familiar with Duolingo's operations and therefore can really relate to user's reviews about the app
3. Regular updates, The app has rolled out regular patch, minor and the occasional major updates.

Below we have code that performs the basic data collection using a popular library for scraping data on google play store.

# Install Packges

In [ ]:
!pip install pandas
!pip install google-play-scraper

# Fetch reviews and store in CSV
To fetch the data we use the given library function and we fetch 60,000 reviews sorted by relevance.

**Note**: We have elected to use relevance as a sorting feature to be able to ensure that we can have a good balance between review types and relevant reviews may have more actionable information.

The returned reviews are plugged into a Data Frame with focus on base columns like ['userName', 'score', 'content', 'at', 'appVersion'].

These columns were picked as they have provided vital information regarding the reviews left by app users.


In [ ]:
from google_play_scraper import reviews, Sort
import pandas as pd

app_id = "com.duolingo"  # Duolingo app ID

def fetch_reviews_by_sort(app_id, sort_type, count=1000):
    result, _ = reviews(
        app_id=app_id,
        lang='en',
        country='us',
        sort=sort_type,
        count=count
    )

    df = pd.DataFrame(result)
    df['sort_type'] = sort_type.name  # Track source
    return df


# Fetch most relevant reviews
df = fetch_reviews_by_sort(app_id, Sort.MOST_RELEVANT, count=60000)

columns =['userName', 'score', 'content', 'at', 'appVersion']

# Keep relevant columns
df = df[columns]

df.to_csv("duolingo_reviews.csv", index=False)

print(df.head())

# Data Cleaning

To clean the data, we have followed the following steps:
1. Removal of duplicate reviews.
2. We remove any data that has a nullable value within any of our base columns.
3. we create a new column cleaned_content that is the content column without the stopwords and any links. We also convert the text into lowercase to standardize it.
4. We once again remove any data points that have nullable cleaned_content.

In [ ]:
# remove duplicates
df.drop_duplicates(subset='content', inplace=True)

# remove null values

for column in columns:
  df = df[df[column].notnull()]

#standardize text

import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_content'] = df['content'].apply(clean_text)

#clean review out that have no cleaned_content
df = df[df["cleaned_content"].notnull()]
df = df[df['cleaned_content'].str.strip() != ""]


#make sure date is in datetime format

df['at'] = pd.to_datetime(df['at'])
dateRange = (df["at"].min(), df['at'].max())
versionRange = (df["appVersion"].min(),df["appVersion"].max())
print( "Time Interval of data: ",dateRange)
print("Number of Data Points: ", len(df))
print("Version Interval of data: ", versionRange)


In [ ]:
!pip install sklearn

# Exploring and Analysing the data

1. Word Frequency:
We look at the word frequency by analysing the 50 most frequent words. We can already understand that the frequency of neutral and positive words will dominate. But we are more interested in understanding the common themes in user reviews.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    stop_words='english',
    max_features=50
)

X = vectorizer.fit_transform(df['cleaned_content'])
word_counts = X.sum(axis=0)
words = vectorizer.get_feature_names_out()

freq_df = pd.DataFrame({
    'word': words,
    'frequency': word_counts.tolist()[0]
}).sort_values(by='frequency', ascending=False)

print(freq_df)

## Word Frequency findings:

1. As expected the words are more over postive and neutral domain. However some themes have begun to emerge.
2. Energy is very common word, contextually implying that a common theme of reviews is the energy feature introduced in version 6 and after.
3. other words like ads and pay imply issues regarding the experieces with these instances of the app.


In [ ]:
from datetime import datetime, timedelta

five_months_ago = datetime.now() - timedelta(days=150)
recent_reviews = df[df['at'] >= five_months_ago]

In [ ]:
df['month'] = df['at'].dt.to_period('M')
monthly_counts = df.groupby('month').size()
print(monthly_counts)

## Monthly reviews count:

1. The reviews have spiked recently as compared to before.
2. The spike is disproportionate with respect to increase in users.
3. The recent updates have excited the users.

In [ ]:
monthly_rating = df.groupby('month')['score'].mean()
print(monthly_rating)

# Sentiment Analysis


To analyze user reviews, the VADER (Valence Aware Dictionary and sEntiment Reasoner) sentiment analysis model was selected due to its suitability for short, informal text data such as mobile application reviews.

VADER is a lexicon and rule-based sentiment analysis tool specifically designed for social media and user-generated content. Unlike traditional machine learning models, it does not require labeled training data, making it computationally efficient and easy to implement for large-scale review datasets.

One of the primary reasons for choosing VADER is its ability to handle linguistic nuances commonly found in app reviews, including:

- Emphasis through capitalization (e.g., "THIS UPDATE IS TERRIBLE")
- Repeated punctuation (e.g., "awful!!!")
- Degree modifiers (e.g., "very bad", "extremely slow")
- Negations (e.g., "not good")
- Informal language and short expressions

VADER produces four sentiment metrics: positive, negative, neutral, and a compound score. The compound score ranges from -1 to +1 and represents the normalized overall sentiment of a review.

In this study, the compound score was used for classification as follows:

- Compound ≥ 0.05 → Positive  
- Compound ≤ -0.05 → Negative  
- Otherwise → Neutral  

The compound score was particularly useful for downstream analysis, including clustering negative reviews and identifying major complaint themes across specific time periods and app version updates.

Given the nature of the dataset—short, emotionally expressive app reviews—VADER provides a reliable, interpretable, and computationally efficient approach for sentiment classification.

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    return analyzer.polarity_scores(text)['compound']

df['sentiment_score'] = df['cleaned_content'].apply(get_sentiment)

def sentiment_label(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['sentiment'] = df['sentiment_score'].apply(sentiment_label)
negative_reviews = df[df['sentiment'] == 'Negative'].copy()
neutral_reviews = df[df['sentiment'] == 'Neutral'].copy()
positive_reviews = df[df['sentiment'] == 'Positive'].copy()



# Distribution of Reviews

Based on the sentiment assigned by our parameters at Vader. The next step is to visualize the reviews and understand any focus regions that can help us understand the negative review themes better.

In [ ]:
import matplotlib.pyplot as plt

# Count sentiment categories
sentiment_counts = df['sentiment'].value_counts()

# Ensure consistent order
sentiment_counts = sentiment_counts.reindex(['Positive', 'Neutral', 'Negative']).fillna(0)

colors = ['green', 'yellow', 'red']

# Create bar plot
plt.figure()
plt.bar(sentiment_counts.index, sentiment_counts.values, color=colors)

plt.xlabel("Sentiment")
plt.ylabel("Number of Reviews")
plt.title("Sentiment Distribution of Reviews")

plt.show()

## Simple Distribution Analysis:

1. The simple count distribution suggests that majority of app users have been in favor of the app updates and experience.
2. A major focus is that there is a lack of Neutral reviews, which means that the average user is either content with the app or incontent.
3. We need to plot it based on version and months to understand the trends better.

In [ ]:
df['at'] = pd.to_datetime(df['at'])

# Extract month (year-month)
df['month'] = df['at'].dt.to_period('M')

# Count of each sentiment per month
monthly_sentiment = df.groupby(['month', 'sentiment']).size().unstack(fill_value=0)

# Ensure consistent order for sentiment columns
monthly_sentiment = monthly_sentiment[['Positive', 'Neutral', 'Negative']]

# Plot
monthly_sentiment.plot(
    kind='bar',
    stacked=True,
    color=['green', 'yellow', 'red'],
    figsize=(20,6)
)

plt.xlabel("Month")
plt.ylabel("Number of Reviews")
plt.title("Monthly Sentiment Distribution")
plt.xticks(rotation=45)
plt.legend(title='Sentiment')
plt.tight_layout()
plt.show()

## Monthly Distribution Analysis:
1. The spike in recent reviewer activity is very evident.
2. What is also to be understood is that recently the number of negative reviews have increased manyfold.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap, Normalize

# Ensure datetime
df['at'] = pd.to_datetime(df['at'])

# Extract month
df['month'] = df['at'].dt.to_period('M')

# Count sentiment per month
monthly_sentiment = df.groupby(['month', 'sentiment']).size().unstack(fill_value=0)
monthly_sentiment = monthly_sentiment[['Positive', 'Neutral', 'Negative']]

# Calculate average rating per month
avg_rating = df.groupby('month')['score'].mean().reindex(monthly_sentiment.index)

# Define gradient colormap: red -> orange -> yellow -> light green -> dark green
colors = ['red', 'orange', '#a88100', 'lightgreen', 'green']
cmap = LinearSegmentedColormap.from_list('rating_cmap', colors)

# Normalize rating for colormap
norm = Normalize(vmin=avg_rating.min(), vmax=avg_rating.max())
rating_colors = [cmap(norm(r)) for r in avg_rating]

# Plot stacked bar
ax = monthly_sentiment.plot(
    kind='bar',
    stacked=True,
    color=['green', 'yellow', 'red'],
    figsize=(20,6),
    width=0.8
)

# Annotate average rating on top with gradient colors
for i, rating in enumerate(avg_rating):
    total_height = monthly_sentiment.iloc[i].sum()
    ax.text(
        i,
        total_height + 2,  # slightly above bar
        f"{rating:.2f}",
        ha='center',
        va='bottom',
        fontsize=6,
        fontweight='bold',
        color=rating_colors[i]
    )

plt.xlabel("Month")
plt.ylabel("Number of Reviews")
plt.title("Monthly Sentiment Distribution with Average Rating Colored by Value")
plt.xticks(rotation=45)
plt.legend(title='Sentiment')
plt.tight_layout()
plt.show()

### Monthly Distribution with Average App rating:
The chart shows the monthly distribution of Positive, Neutral, and Negative reviews using a stacked bar format. Each bar represents total review volume, while segment sizes indicate the proportion of each sentiment category. The average monthly rating is displayed above each bar and color-coded from red (lower ratings) to green (higher ratings) for quick visual interpretation. Overall, the plot highlights shifts in user sentiment and provides a clear view of how overall ratings align with sentiment trends over time.

Points of focus:
1. Recently the app has recieved some heavy criticisms.
2. these criticisms are consistent with both sentiment analysis and trends in the rating of the app.
3. highest impact is between 2025-07 to 2026-02.
4. Let us focus more in this region.

In [ ]:
# Extract minor version (major + minor)
df['major_version'] = df['appVersion'].str.split('.').str[0]

# Count sentiment per minor version
minor_sentiment = df.groupby(['major_version', 'sentiment']).size().unstack(fill_value=0)
minor_sentiment = minor_sentiment[['Positive', 'Neutral', 'Negative']]
print(df.head())
# Sort numerically
minor_sentiment = minor_sentiment.sort_index(
    key=lambda x: x.map(lambda y: [int(i) for i in y.split('.')])
)

# Plot
minor_sentiment.plot(
    kind='bar',
    stacked=True,
    color=['green', 'yellow', 'red'],
    figsize=(16,6),
    width=0.8
)

plt.xlabel("Major Version")
plt.ylabel("Number of Reviews")
plt.title("Sentiment Distribution by Major Version")
plt.legend(title='Sentiment')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

### Version Distribution Analysis:
1. version 6 has recieved the most negative reviews.
2. This ratio suggests that the new updates to the UX have impacted some users in an undesired manner.
3. The reason for selecting major version updates over minor or patch is that as per Duo-lingo's documentation, they only make major shifts to the UX in major versions.


In [ ]:
df['at'] = pd.to_datetime(df['at'])

# Filter negative reviews
negative_reviews = df[df['sentiment'] == 'Negative'].copy()

# Define date range
start_date = pd.Timestamp('2025-07-01')
end_date   = pd.Timestamp('2026-02-28')

# Filter by date
negative_reviews_range = negative_reviews[
    (negative_reviews['at'] >= start_date) & (negative_reviews['at'] <= end_date)
]

# Sort by sentiment_score ascending (most negative first)
negative_reviews_range_sorted = negative_reviews_range.sort_values(by='sentiment_score', ascending=True)

negative_reviews_range_sorted.to_csv("duolingo_reviews_negative_spike.csv", index=False)


### Focus Region
Here the focus region is updated to include only Negative reviews from the desired region.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vectorize reviews
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(negative_reviews_range_sorted['cleaned_content'])

# Compute similarity between all reviews
similarity_matrix = cosine_similarity(X)

# Example: find reviews similar to review #0
similar_indices = similarity_matrix[0].argsort()[::-1][1:6]  # top 5 similar reviews
similar_reviews = negative_reviews_range_sorted.iloc[similar_indices]

print(similar_reviews)

## 🔍 Clustering & Thematic Analysis of Negative Reviews

### 1. Text Extraction
- Extract cleaned negative review texts from the DataFrame.
- Convert them into a list for embedding generation.

### 2. Sentence Embedding (Semantic Representation)
- Use Sentence-BERT (`all-MiniLM-L6-v2`) to convert each review into dense vector embeddings.
- Embeddings capture contextual semantic meaning beyond simple keyword matching.

### 3. K-Means Clustering
- Apply K-Means clustering with k = 5 to group semantically similar reviews.
- Assign each review a cluster label and append it to the dataset.

### 4. Top 3 Dominant Clusters Selection
- Count reviews per cluster using `Counter`.
- Select the top 3 clusters with the highest number of reviews for focused analysis.

### 5. Cluster-Level Theme Extraction
For each of the top 3 clusters:
- Compute TF-IDF to extract top 10 representative keywords.
- Calculate the cluster centroid in embedding space.
- Measure cosine similarity between each review and the centroid.
- Select the top 3 most representative reviews (closest to centroid).
- Truncate long reviews for concise reporting.

### 6. Cluster Summary Generation
- Combine keywords and representative reviews into a structured summary.
- Display cluster size, key themes, and sample reviews.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1️⃣ Extract cleaned review texts
texts = negative_reviews_range_sorted['cleaned_content'].tolist()

# 2️⃣ Create embeddings using Sentence-BERT
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts, show_progress_bar=True)

# 3️⃣ Cluster embeddings
num_clusters = 5  # try 5 clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(embeddings)

# Add cluster labels to DataFrame
negative_reviews_range_sorted = negative_reviews_range_sorted.reset_index(drop=True)
negative_reviews_range_sorted['cluster'] = labels

# 4️⃣ Find top 3 clusters by number of reviews
cluster_counts = Counter(labels)
top3_clusters = [cluster for cluster, _ in cluster_counts.most_common(3)]
print("Top 3 clusters:", top3_clusters)

# 5️⃣ Extract representative reviews & summary per cluster
cluster_summaries = []

for cluster in top3_clusters:
    cluster_df = negative_reviews_range_sorted[negative_reviews_range_sorted['cluster']==cluster]
    cluster_texts = cluster_df['cleaned_content'].tolist()

    # TF-IDF top keywords for summary
    tfidf = TfidfVectorizer(stop_words='english', max_features=10)
    tfidf.fit(cluster_texts)
    keywords = tfidf.get_feature_names_out()

    # Compute cluster centroid and similarity to centroid
    cluster_embeddings = embeddings[cluster_df.index]
    centroid = cluster_embeddings.mean(axis=0)
    sim_scores = cosine_similarity(cluster_embeddings, centroid.reshape(1, -1)).flatten()

    # Pick top 3 most representative reviews
    top_indices = sim_scores.argsort()[::-1][:3]
    representative_reviews = [cluster_texts[i] for i in top_indices]

    # Optional: truncate long reviews to first 2 sentences
    truncated_reviews = ['. '.join(r.split('.')[:2]) + '.' for r in representative_reviews]

    # Combine keywords + sample reviews for a brief summary
    summary = f"Cluster {cluster}: Keywords: {', '.join(keywords)}. Sample representative reviews: {truncated_reviews}"

    cluster_summaries.append({
        'cluster': cluster,
        'num_reviews': len(cluster_df),
        'keywords': keywords,
        'representative_reviews': truncated_reviews,
        'summary': summary
    })

# 6️⃣ Display summaries
for cs in cluster_summaries:
    print("\n" + "-"*80)
    print(f"Cluster {cs['cluster']} | Number of reviews: {cs['num_reviews']}")
    print("Keywords:", ', '.join(cs['keywords']))
    print("Representative reviews:")
    for r in cs['representative_reviews']:
        print("-", r)
    print("Summary:")
    print(cs['summary'])

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# Generate word clouds only for top 3 clusters
for cluster in top3_clusters:

    cluster_df = negative_reviews_range_sorted[
        negative_reviews_range_sorted['cluster'] == cluster
    ]

    cluster_texts = cluster_df['cleaned_content'].astype(str).tolist()
    combined_text = " ".join(cluster_texts)

    # Optional: add custom stopwords
    custom_stopwords = set(STOPWORDS)
    custom_stopwords.update(["app", "use", "get"])  # add domain-generic words if needed

    # Create word cloud
    wordcloud = WordCloud(
        width=900,
        height=450,
        background_color='white',
        stopwords=custom_stopwords,
        max_words=100
    ).generate(combined_text)

    # Plot one figure per cluster
    plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Word Cloud - Negative Review Cluster {cluster}")
    plt.show()

# Key Insights from Negative Reviews

### 1. Flaws in the Energy System (Version 6+)
A large portion of negative reviews highlight issues with the energy system. Free users frequently report that their progress is blocked due to depleted energy, leading to frustration and a feeling of stalled learning.

### 2. AI Recommendations Are Problematic
Users criticize the AI-driven lesson recommendations, reporting low accuracy and repetitive lesson suggestions. Many feel the AI does not adapt properly to their learning needs, resulting in a frustrating experience.

### 3. Excessive Advertising Interruptions
Many users find the frequency of ads disruptive. Unlike entertainment apps, Duolingo users are here to learn, so the ads interrupt their focus. This issue is often compounded by the energy system, as users feel forced to watch ads to continue progress.

# Suggested App Improvements

### 1. Revise the Energy System
- Adjust the energy system so that energy is only depleted when mistakes are made, not simply by completing lessons.  
- Increase energy limits for free users or provide alternative ways to earn energy through daily activities.  
- Introduce a smoother progression system that reduces frustration from blocked lessons.  
- Offer clear communication about energy consumption and refill options.

### 2. Improve AI Lesson Recommendations
- Enhance the AI algorithm to reduce repetition and better adapt to individual learning pace.  
- Allow users to provide feedback on recommended lessons to refine AI suggestions.  
- Introduce optional “lesson variety” settings so users can control lesson rotation.  
- Add a setting for users to turn off AI recommendations entirely if they prefer.

### 3. Reduce Advertising Disruption
- Limit ad frequency for free users, especially when energy has been depleted.  
- Provide a clear distinction between learning and optional ad engagement.  
- Explore non-intrusive ad formats or incentives (e.g., optional ads for extra energy rather than forced interruptions).